<a href="https://colab.research.google.com/github/mmanteli/Applications-of-Data-Analysis-2021/blob/main/ADA2021_exercise2_AmandaMyntti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Amanda Myntti <br>
Student number 514011 <br>
February, 10th, 2021  <br>

# Exercise 2 | TKO_2096 Application of Data Analysis 2021

#### Prediction of the metal ion content from multi-parameter data <br>
- Use K-Nearest Neighbor Regression with euclidean distance to predict total metal concentration (c_total), concentration of Cadmium (Cd) and concentration of Lead (Pb), for each sample using number of neighbors k = 3.<br> <br>

    - You may use Nearest Neighbor Regression from https://scikit-learn.org/stable/modules/neighbors.html
    - The data should be standarized using z-score.
    - Implement your own Leave-One-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb). 
    - Implement your own Leave-Replicas-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb).
    - Return your solution as a Jupyter Notebook file (include your full name in the file name).
    - Submit to moodle your solution on **Wednesday 10 of February** at the latest.

### Comments by the student
This was a nice exercise, and it was much easier to do than the 1st one, simply because there we're more instructions on the template. I learned a lot, and I found a better way to do splits than in the last exercise! :) 

There are a few things I'd like to point out about my code:

*   My computer died on Friday and I lost all my progress (I had done about 30-40%) and I had to start again on Monday with my new laptop. Luckily I did finish this on time, but now I do not have time to optimize the code at all and the leave-replicas-out algorithm is *really* slow. There might be some spaghetty parts in the code, too. Sorry about that!
*   Also for the same reason I had to copy a lot of my old code, meaning I use the terms "training and validation sets" in the code, but speak about "training and test sets". I do not have time (or do I dare) to make changes in the code anymore, since if I break something now, I'll never correct in on time! So for all of this, consider validation == test.



## Import libraries

In [ ]:
#In this cell import all libraries you need. For example: 
import numpy as np
import pandas as pd
from mlxtend.preprocessing import standardize
from sklearn.neighbors import KNeighborsRegressor

## Data preprosessing
I'll first take a look at the data, find replicas, and z-score standardize the features I'll be using.

### Read and visualize the dataset

In [ ]:
# read the data
water_data = pd.read_csv('Water_data.csv')

# printing info, because it gives us more info than the size of the dataframe
water_data.info()

# printing the first five rows
water_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   c_total  225 non-null    int64  
 1   Cd       225 non-null    float64
 2   Pb       225 non-null    float64
 3   Mod1     225 non-null    int64  
 4   Mod2     225 non-null    int64  
 5   Mod3     225 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 10.7 KB


,c_total,Cd,Pb,Mod1,Mod2,Mod3
0,2000,800.0,1200.0,126430,2604,6996
1,35,14.0,21.0,20597,271,138677
2,35,14.0,21.0,24566,269,161573
3,35,35.0,0.0,105732,971,132590
4,100,20.0,80.0,57774,5416,93798


#### To show understanding of the data, answer the following questions:
- How many different mixtures of Cadmium (Cd) and Lead (Pb) were measured? <br>
- How many total concentrations (c_total) were measured? <br>
- How many mixtures have less than 4 replicas? <br>
- How many mixtures have 4 or more replicas? Print out c_total, Cd and Pb for those concentrations.<br>

In [ ]:
# print the length of the whole data to get the number of all measurements
print("Number of total entries: ",len(water_data['c_total']))

# To get the number of unique measurements, I'll make another dataframe that has 
# all the unique measurements in one row, and the last column is the fequency
water_data_freq = water_data.groupby(['c_total','Cd', 'Pb']).size().reset_index(name='Freq')

# can be printed if needed
#print("Data according to frequency:")
#display(water_data_freq)

# the length of this dataframe is the number of unique measurements
print("Distinct measurements of Cd and Pb: ", len(water_data_freq))

# define a few sub dataframes based on the frequency
water_low_freq = water_data_freq[water_data_freq.Freq < 4]
water_high_freq = water_data_freq[water_data_freq.Freq >= 4]

# print the number according to their lengths => number of measurement with  
# corresponding number of replicas
print("Number of measurements with less than 4 replicas: ",len(water_low_freq) )
print("Number of measurements with 4 or more replicas: ",len(water_high_freq) )

# lastly, print out the high freq values
print("") # for clarity
print("Values for high frequency replicas:")
display(water_high_freq)

# I printed this too to see their frequencies, all 3
#display(water_low_freq)


Number of total entries:  225
Distinct measurements of Cd and Pb:  67
Number of measurements with less than 4 replicas:  43
Number of measurements with 4 or more replicas:  24

Values for high frequency replicas:


,c_total,Cd,Pb,Freq
19,50,0.0,50.0,4
20,50,10.0,40.0,4
21,50,20.0,30.0,4
22,50,30.0,20.0,4
23,50,40.0,10.0,4
24,50,50.0,0.0,4
25,70,0.0,70.0,4
26,70,14.0,56.0,4
27,70,28.0,42.0,4
28,70,42.0,28.0,4


From this analysis it is easy to see that there are two kinds of datapoints: some have 3 replicas (43 points) and others 4 (24 points). This equals the number of all datapoints, 43x3 + 4x24 = 225. 

### Standardization of the dataset

In [ ]:
# I used the ready made Z-score standardisation from mlxtend.preprocessing
water_data[['Mod1','Mod2','Mod3']] = standardize(water_data[['Mod1','Mod2','Mod3']])

# display the first 5 values to make sure everything worked
display(water_data.head())

,c_total,Cd,Pb,Mod1,Mod2,Mod3
0,2000,800.0,1200.0,0.166505,-0.508756,-1.499041
1,35,14.0,21.0,-0.892616,-0.701641,0.685861
2,35,14.0,21.0,-0.852896,-0.701806,1.065760
3,35,35.0,0.0,-0.040629,-0.643767,0.584863
4,100,20.0,80.0,-0.520568,-0.276268,-0.058789


### Separating the data
Lastly, I'll separate the data to features (X) and labels (y). We're trying to predict total, Cd and Pb concentrations from the measurements Mod1, Mod2 and Mod3. 


In [ ]:
# features (Mod1, Mod2, Mod3)
X = water_data[['Mod1','Mod2','Mod3']]


# labels (Cd, Pb)
y = water_data[['c_total','Cd','Pb']]


## Algorithm for the Concordance index and data splitting

Next I'll define a function for the concordance index, that  measures if the model was able to rank the data points in the same order as the true outputs. Also train-test splits for leave-one-out CV and leave-replicas-out CV are defined here.

### C-index code 

In [ ]:
def cindex(true_labels, pred_labels):
  """Returns C-index between true labels and predicted labels"""  
  
  # set data collection: concordant pairs and all pairs
  N = 0
  counts = 0.0

  # go over all the pairs (a triangle):

  for i in range(len(true_labels)):
    for j in range(i+1, len(pred_labels)):

      # if the true labels are not the same 
      if (true_labels[i] != true_labels[j]):

        # add 1 to one pairs
        counts +=1.0

        # make the comparison given in the extra material, N += 1 if it's true
        if (true_labels[i] < true_labels[j] and pred_labels[i] < pred_labels[j]) or (true_labels[j] < true_labels[i] and pred_labels[j] < pred_labels[i]):
          N += 1
        
  # c index is the concordant pairs / all pairs
  cindx = N/counts

  return cindx

In [ ]:
#test cindex function with following values
#true_labels = [-1, 1, 1, -1, 1]
#predictions = [0.60, 0.80, 0.75, 0.75, 0.70]

# testing with the extra material data, answer should be 0.67
# I tested with this because I know the correct answer to this
true_labels = [7.1,2.3,5.4,1.8]
predictions = [7.4,3.4,8.5,3.9]

cindx = cindex(true_labels, predictions)
print(cindx)

0.6666666666666666


The c-index function gives the correct result with the data provided in the extra material.

### Splitting functions
These functions are really similar, but are written quite differently. Also, the LRO-function is really slow, since it makes new dataframes everytime. I tried to reduce the runtime by simply modifying each dataframe, not creating a new one all the time, but that lead to an error, and I do not have time debug it now. The function works just like I intended it to, it is simply slow.

In [ ]:
#Include here all the functions that you need to run in the data analysis part.

def LOO_folds(X,y,idx):
  """ 
  Splits the given data into training and validation sets, with |val| = 1.
  Input: features X, labels y, index of LOO idx
  Returns: X_train, X_val, y_train, y_val
  """
  # for training sets, simply drop the one row
  X_train = X.drop(idx)
  y_train = y.drop(idx)

  # for validation sets, drop everything else but the one row
  # make a list to help
  lista = np.arange(0,len(X))
  lista2 = np.delete(lista, idx)
  # drop indices on the list
  X_val = X.drop(lista2)
  y_val = y.drop(lista2)

  # return the sets
  return X_train, X_val, y_train, y_val


def LRO_folds(X,y,idx,droplist):
  """ 
  Splits the given data into training and validation sets so that all replicas
  of the datapoint are in the same validation set.
  Input: features X, labels y, index of first datapoint, 
  droplist = list of indices that have alreafy been in a validation set
  Returns: X_train, X_val, y_train, y_val, updated droplist
  """

  # save values on the idx row
  c_tot = y.iat[idx,0]
  cd = y.iat[idx,1]
  pb = y.iat[idx,2]
  
  # copy X and y to train and val sets
  X_train = X
  X_val = X
  y_train = y
  y_val = y

  # loop over all the points
  for i in range(len(X)):
    # if there's a replica in index i
    if (y.iat[i,0] == c_tot and y.iat[i,1] == cd and y.iat[i,2] == pb):
      # drop it from training data
      X_train = X_train.drop(i)
      y_train = y_train.drop(i)
      # add the index to droplist, so that we'll not redo it
      droplist.append(i)
    else:
      # it's not a replica; drop it from validation
      X_val = X_val.drop(i)
      y_val = y_val.drop(i)

  # return the sets
  return X_train, X_val, y_train, y_val,droplist


## Results for Leave-One-Out cross-validation
Here I'll make predictions to each LOO-split, and after that find the concordance index between the predicted labels and the real labels.

In [ ]:
# make vectors to store data (name accordingly)
c_total_predicted_labels = []
cd_predicted_labels = []
pb_predicted_labels = []

c_total_true_labels = []
cd_true_labels = []
pb_true_labels = []


# loop over all the data
for i in range(len(water_data)):

  # make train-test split, in the test(here val) data there is only 1 point
  X_train, X_val, y_train, y_val = LOO_folds(X,y,i)

  # train knn with 3 neighbors and euclidean metric
  knn = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
  # fit the train data
  knn.fit(X_train, y_train)
  # make a prediction for X_val
  prediction = knn.predict(X_val)

  # append the prediction into data collection vectors
  c_total_predicted_labels.append(prediction[0][0])
  cd_predicted_labels.append(prediction[0][1])
  pb_predicted_labels.append(prediction[0][2])
  # append the true labels (y_val) to data collection vectors
  c_total_true_labels.append(y_val.iat[0,0])
  cd_true_labels.append(y_val.iat[0,1])
  pb_true_labels.append(y_val.iat[0,2])

# calculate the concordance index for all vector pairs
c_index_c_total = cindex(c_total_true_labels, c_total_predicted_labels)
c_index_cd = cindex(cd_true_labels, cd_predicted_labels)
c_index_pb = cindex(pb_true_labels, pb_predicted_labels)

# print them
print("Concordance index for c_total: ", c_index_c_total)
print("Concordance index for Cd: ",c_index_cd)
print("Concordance index for Pb: ",c_index_pb)


Concordance index for c_total:  0.9099991312657458
Concordance index for Cd:  0.8961456549880987
Concordance index for Pb:  0.8716749488453669


## Results for Leave-Replicas-Out cross-validation
This time I'll use a different way to split the data, in hopes of getting more reliable results. I'll make splits so that all replicas are in one validation set, and then make predictions for all of them at the same time. Concordance index is calculated the same way as in LOO.

In [ ]:
# we'll do exactly the same thing as before, but with different splits
# and we make more thanone prediction each time, either 3 or 4
# depending on the data point (how many replicas it has)

# make data collection vectors
c_total_predicted_labels = []
cd_predicted_labels = []
pb_predicted_labels = []

c_total_true_labels = []
cd_true_labels = []
pb_true_labels = []

# droplist tells us if we have already included a datapoint in a
# validation set; we want each datapoint to be in one validation set,
# so that we do not make multiple predictions for one datapoint
# droplist is updated everytime we make a train-test split
droplist = []

# loop over data
for i in range(len(water_data)):
  
  # if this index has not been in a validation set yet
  if i not in droplist:
    
    # make split with LRO folds
    X_train, X_val, y_train, y_val, droplist = LRO_folds(X,y,i,droplist)

    # train knn similarly to before, fit, and predict for X_val
    knn = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_val)
    
    # append all 3 or 4 predictions to the data collection vectors
    c_total_predicted_labels.extend(prediction[:,0])
    cd_predicted_labels.extend(prediction[:,1])
    pb_predicted_labels.extend(prediction[:,2])

    # here the same but with the real labels
    c_total_true_labels.extend(y_val['c_total'].tolist())
    cd_true_labels.extend(y_val['Cd'].tolist())
    pb_true_labels.extend(y_val['Pb'].tolist())


# calculate the concordance index similarly as before
c_index_c_total = cindex(c_total_true_labels, c_total_predicted_labels)
c_index_cd = cindex(cd_true_labels, cd_predicted_labels)
c_index_pb = cindex(pb_true_labels, pb_predicted_labels)

# print results
print("Concordance index for c_total: ", c_index_c_total)
print("Concordance index for Cd: ", c_index_cd)
print("Concordance index for Pb: ", c_index_pb)


Concordance index for c_total:  0.8091390843540961
Concordance index for Cd:  0.7578402305090408
Concordance index for Pb:  0.7649809997076878


## Interpretation of results
#### Answer the following questions based on the results obtained
- Which cross-validation approach had more optimistic results?
- Which cross-validation generalize better on unseen data? Why?

More optimistic results were given by LOO:

Concordance index for c_total:  0.9099991312657458 \
Concordance index for Cd:  0.8961456549880987 \
Concordance index for Pb:  0.8716749488453669. \

With LRO, the results were about 10 %-points lower:

Concordance index for c_total:  0.8091390843540961 \
Concordance index for Cd:  0.7578402305090408 \
Concordance index for Pb:  0.7649809997076878. \

These values seem to be really close to the values obtained in the lecture material (the graphs with k = 3 seem to have similar values).

When we do cross validation with LOO, replicated points in the data set cause data leakage. While we have a quite similar (same labels, almost same Mod1-3 values) in the training set, making predictions on the test set is much easier compared to the situation, where the test set is completely unseen data. We avoid this problem with LRO, because similar data points are in the same test set. Thus LRO cross validation generalizes better on unseen data.